In [3]:
import pandas as pd

df = pd.read_csv("deanonymous_score_10000.csv")

df["confidence"] = 0.5
df["trustiness"] = 0.5
df["reliability"] = 0.7

max_iterations = 10000
epsilon = 0.01

for iteration in range(max_iterations):
    df["trustiness_prev"] = df["trustiness"]
    df["reliability_prev"] = df["reliability"]
    df["confidence_prev"] = df["confidence"]
    df["weighted_score"] = df["deanonymous"] * df["confidence"]
    
    trustiness_updates = df.groupby("to")["weighted_score"].mean()
    df["trustiness"] = df["to"].map(trustiness_updates)

    reliability_updates = df.groupby("from")["confidence"].mean()
    df["reliability"] = df["from"].map(reliability_updates)

    df["trustiness"] = df["trustiness"].clip(0, 1)
    df["reliability"] = df["reliability"].clip(0, 1)

    df["confidence"] = (df["reliability"] + (1 - abs(df["deanonymous"] - df["trustiness"]))) / 2

    df["confidence"] = df["confidence"].clip(0, 1)

    delta_t = (df["trustiness"] - df["trustiness_prev"]).abs().sum()
    delta_r = (df["reliability"] - df["reliability_prev"]).abs().sum()
    delta_c = (df["confidence"] - df["confidence_prev"]).abs().sum()
    delta = max(delta_t, delta_r, delta_c)

    print(f"Iteration {iteration + 1}: ΔT = {delta_t:.4f}, ΔR = {delta_r:.4f}, ΔC = {delta_c:.4f}, Δ = {delta:.4f}")

    if delta < epsilon:
        print("Convergence achieved!")
        break

df["risk"] = (1 - df["reliability"]) * 10

Iteration 1: ΔT = 1409970.5664, ΔR = 827292.8000, ΔC = 623668.2500, Δ = 1409970.5664
Iteration 2: ΔT = 95516.3961, ΔR = 615895.4234, ΔC = 347636.8958, Δ = 615895.4234
Iteration 3: ΔT = 77731.8244, ΔR = 347573.5742, ΔC = 206624.9883, Δ = 347573.5742
Iteration 4: ΔT = 58930.5660, ΔR = 206564.9477, ΔC = 128822.3229, Δ = 206564.9477
Iteration 5: ΔT = 44981.9077, ΔR = 128763.3629, ΔC = 84551.1506, Δ = 128763.3629
Iteration 6: ΔT = 34922.3885, ΔR = 84487.0247, ΔC = 58465.1335, Δ = 84487.0247
Iteration 7: ΔT = 27598.5021, ΔR = 58401.6510, ΔC = 42317.2408, Δ = 58401.6510
Iteration 8: ΔT = 22159.8142, ΔR = 42263.6662, ΔC = 31887.2540, Δ = 42263.6662
Iteration 9: ΔT = 18068.2492, ΔR = 31843.9971, ΔC = 24779.8703, Δ = 31843.9971
Iteration 10: ΔT = 14933.4471, ΔR = 24747.4040, ΔC = 19723.9389, Δ = 24747.4040
Iteration 11: ΔT = 12474.2657, ΔR = 19697.5991, ΔC = 16005.8789, Δ = 19697.5991
Iteration 12: ΔT = 10515.9299, ΔR = 15981.4860, ΔC = 13202.0466, Δ = 15981.4860
Iteration 13: ΔT = 8940.8134, ΔR

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 1]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [5]:
len(merged_df[merged_df['risk'] < 6])

52

In [6]:
len(merged_df[merged_df['risk'] >= 6])

183

In [7]:
len(merged_df[merged_df['risk'].isna()])

8

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 0]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [9]:
len(merged_df[merged_df['risk'] < 6])

478

In [10]:
len(merged_df[merged_df['risk'] >= 6])

58

In [11]:
len(merged_df[merged_df['risk'].isna()])

24